In [72]:
import pyvisa as visa

In [1]:
from qcodes_contrib_drivers.drivers.QDevil import QDAC2
from time import sleep
qdac_addr = '192.168.0.198'
qdac = QDAC2.QDac2('QDAC', visalib='@py', address=f'TCPIP::{qdac_addr}::5025::SOCKET')

Connected to: QDevil QDAC-II (serial:200, firmware:11-1.21) in 0.16s


# set DC

In [2]:
#qdac.ch01.output_mode(range='low', filter='dc')
#qdac.ch01.dc_constant_V(0)
qdac.ch02.output_mode(range='low', filter='dc')
qdac.ch02.dc_constant_V(0.1)
qdac.errors()

'0, "No error"'

# set sine wave

In [3]:
qdac.reset()
sine = qdac.ch02.sine_wave()
sine.start()

# 1D sweep

In [9]:
with qdac.ch02.dc_sweep(
    repetitions=2,
    start_V=-1,
    stop_V=1,
    points=11,
    dwell_s=10e-6
) as my_list:
    print(my_list.points())

11


ch2 :1d sweep ch3: trigger

In [10]:
qdac.reset()

In [ ]:
repeat = 100000
dc_ch2 = qdac.ch02.dc_sweep(
    repetitions=repeat,
    stepped=True,
    start_V=-1,
    stop_V=1,
    points=11,
    dwell_s=100e-6,
)
square_ch3 = qdac.ch03.square_wave(frequency_Hz=10000, repetitions=repeat*11)
qdac.start_all()

11

ch2 1d sweep trig out1: tigger

In [23]:
import numpy as np

In [28]:
qdac.reset()

# 2. 準備 DC sweep（例：-0.4 V → +0.4 V，共 101 點，dwell 200 µs）
voltages = np.linspace(-0.4, 0.4, 101)
dwell_s  = 100e-6                       # 200 µs ⇒ 5 kHz

dc_ch2 = qdac.ch02.dc_sweep(
    repetitions=repeat,
    stepped=True,
    start_V=-1,
    stop_V=1,
    points=11,
    dwell_s=100e-6,
)  # −1 = 連續 loop

# 3. 取出「每一次 dwell 結束」的內部觸發
step_trig = dc_ch2.step_end_marker()       # ➜ QDac2Trigger_Context 物件 :contentReference[oaicite:0]{index=0}

# 4. 將內部觸發接到 Trigger OUT 1，寬度 = ½ 週期
qdac.ext1.width_s(dwell_s / 2)                # 50 % duty
qdac.ext1.polarity('norm')                    # 正邏輯：高電平為「1」
qdac.ext1.source_from_trigger(step_trig)  
qdac.ext1.delay_s(0.1*dwell_s)    # routing :contentReference[oaicite:1]{index=1}

# 5. Arm 並啟動
qdac.start_all()                              # List generator + 外部觸發同時就緒


In [25]:
step_trig = dc_ch2.step_end_marker()       # ➜ QDac2Trigger_Context 物件 :contentReference[oaicite:0]{index=0}


In [12]:
dc_sweep.start()

In [27]:
qdac.ext1?

Type:        QDac2ExternalTrigger
String form: <QDac2ExternalTrigger: QDAC_ext1 of QDac2: QDAC>
File:        /opt/anaconda3/envs/qiskit_env_new/lib/python3.9/site-packages/qcodes_contrib_drivers/drivers/QDevil/QDAC2.py
Docstring:  
External output trigger

There are three 5V isolated triggers on the front (1, 2, 3) and two
non-isolated 3V3 on the back (4, 5).